# 6.- Azure ML Environment Definition

In [22]:
import yaml

from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, BuildContext

## Define Variables

In [23]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [24]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

environment_name = config["azure"]["environment_name"]

## Azure Authentication

In [25]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

In [26]:
# Update Azure ML Client
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


## Environment and Context

In [27]:
def create_and_verify_environment(ml_client, environment_name, docker_file_path):
    """
    Creates a new Azure ML environment from a Docker context if it does not already exist.

    This function first checks if an environment with the specified name and label "latest" exists.
    If it does, it prints a message and returns without creating a new environment.
    Otherwise, it builds a new environment using the provided Docker context.

    Parameters:
        ml_client (MLClient): The Azure ML client used to manage environments.
        environment_name (str): The name of the environment.
        docker_file_path (str): The path to the Docker context directory containing the Dockerfile.
            This should be the directory containing the Dockerfile, not the Dockerfile itself.

    Returns:
        None
    """
    try:
        existing_environment = ml_client.environments.get(
            name=environment_name,
            label="latest"
        )
        print(f"Environment '{environment_name}' already exists. No need to recreate.")
        return
    except Exception as e:
        print(f"Environment '{environment_name}' not found. Creating a new one.")

    build_context = BuildContext(
        path=docker_file_path
    )

    env_docker_context = Environment(
        build=build_context,
        name=environment_name,
        description="Environment created from a Docker context.",
        tags={
            "project": "YOLO 11",}
        )

    try:
        created_env = ml_client.environments.create_or_update(env_docker_context)
        print(f"Environment '{created_env.name}' created.")
    except Exception as e:
        print(f"Failed to create or update environment: {e}")


In [ ]:
docker_file_path = "../docker"
create_and_verify_environment(ml_client, environment_name, docker_file_path)